# Aggregated BiTE profiles

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import re
import os
import sys
import h5py
import json
from sklearn.manifold import TSNE


In [ ]:
annot_df = pd.read_csv('../data/BiTE/bite_annot.csv')

In [ ]:
prof_dir = '../data/bite_mean_profiles/'

In [ ]:
plates = os.listdir(prof_dir)

In [ ]:
data_all = []
for plate in plates:
    imgdf = pd.read_csv(os.path.join(prof_dir, plate))
    annot_plate = annot_df[annot_df['plate']==plate.replace('.csv', '')]
    plate_df = pd.merge(imgdf, annot_plate[['well', 'PatientID', 'comb']],on='well')
    plate_df = plate_df.groupby(['PatientID', 'comb'], as_index=False).agg('mean')
    plate_df.index = plate_df['comb']
    plate_df = plate_df.drop('comb', axis=1)
    data_all.append(plate_df)

In [ ]:
df_all = pd.concat(data_all)

In [ ]:
df_all

In [ ]:
df_wide = pd.pivot_table(df_all, index=df_all.index, columns='PatientID')

In [ ]:
df_wide.shape

In [ ]:
from bioimg.singlecell import plot_heatmap
plot_heatmap(df_wide.fillna(0), xticklabels=False, size=(10,14),
            yticklabels=True, vmin=-3, vmax=3)

In [ ]:
df_wide.columns = df_wide.columns.map('|'.join)

In [ ]:
df_wide.head()

In [ ]:
df_wide.to_csv('../data/all_bite_profiles.csv')

Collapse concentrations:

In [ ]:
combs = annot_df.apply(lambda x: '_'.join([x['BiTE'], x['Drug']]), axis=1).values

In [ ]:
combs[combs=='DMSO_DMSO'] = 'DMSO'
combs = [c.replace("_DMSO", '') for c in combs]
combs = [c.replace("DMSO_", '') for c in combs]

In [ ]:
annot_df['comb'] = combs

In [ ]:
annot_df.head()

In [ ]:
data_all = []
for plate in plates:
    imgdf = pd.read_csv(os.path.join(prof_dir, plate))
    annot_plate = annot_df[annot_df['plate']==plate.replace('.csv', '')]
    plate_df = pd.merge(imgdf, annot_plate[['well', 'PatientID', 'comb']],on='well')
    # take extreme values
    plate_df = (plate_df.
                groupby(['PatientID', 'comb'],
                        as_index=False).
                agg(lambda x: np.max(x) if np.abs(np.max(x)) > np.abs(np.min(x)) else np.min(x)))
    plate_df.index = plate_df['comb']
    plate_df = plate_df.drop('comb', axis=1)
    data_all.append(plate_df)

In [ ]:
df_all = pd.concat(data_all)
df_wide = pd.pivot_table(df_all, index=df_all.index, columns='PatientID')
df_wide.columns = df_wide.columns.map('|'.join)

In [ ]:
df_wide.to_csv('../data/bite_comb_profiles.csv')